In [12]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

from main import client

In [13]:
class Agent:
    def __init__(self, system):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(model="gpt-4-0125-preview", temperature=0, messages=self.messages)
        return completion.choices[0].message.content

In [14]:
prompt = """
    You run in a loop of Thought, Action, PAUSE, Observation.
    At the end of the loop you output an Answer
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.

    Your available actions are:

    calculate:
    e.g. calculate: 4 * 7 / 3
    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

    average_dog_weight:
    e.g. average_dog_weight: Collie
    returns average weight of a dog when given the breed

    Example session:

    Question: How much does a Bulldog weigh?
    Thought: I should look the dogs weight using average_dog_weight
    Action: average_dog_weight: Bulldog
    PAUSE

    You will be called again with this:

    Observation: A Bulldog weights 51 lbs

    You then output:

    Answer: A bulldog weights 51 lbs
    """.strip()

In [15]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if "Scottish Terrier" in name:
        return "Scottish Terriers average 20 lbs"
    elif "Border Collie" in name:
        return "a Border Collies average weight is 37 lbs"
    elif "Toy Poodle" in name:
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"
    
known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}


In [16]:
abot = Agent(prompt)
result = abot("How much does a toy poodle weigh?")
print(result)
result = average_dog_weight(" Toy Poodle")
print(result)
next_prompt = f"Observation: {result}"
result = abot(next_prompt)
print(result)

Thought: I should look up the dog's weight using average_dog_weight for a Toy Poodle.
Action: average_dog_weight: Toy Poodle
PAUSE
a toy poodles average weight is 7 lbs
Answer: A Toy Poodle weighs an average of 7 lbs.


In [17]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\n    At the end of the loop you output an Answer\n    Use Thought to describe your thoughts about the question you have been asked.\n    Use Action to run one of the actions available to you - then return PAUSE.\n    Observation will be the result of running those actions.\n\n    Your available actions are:\n\n    calculate:\n    e.g. calculate: 4 * 7 / 3\n    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\n    average_dog_weight:\n    e.g. average_dog_weight: Collie\n    returns average weight of a dog when given the breed\n\n    Example session:\n\n    Question: How much does a Bulldog weigh?\n    Thought: I should look the dogs weight using average_dog_weight\n    Action: average_dog_weight: Bulldog\n    PAUSE\n\n    You will be called again with this:\n\n    Observation: A Bulldog weights 51 lbs\n\n    You then output:\n\n    An

### Let's move with more complex Task


In [25]:
abot = Agent(prompt)

In [26]:
question = """
I have 2 dogs, a border collie and a scottish terrier.\
What is their combined weight.
""".strip()
abot(question)

'Thought: To find the combined weight of the two dogs, I need to first find the average weight of a Border Collie and a Scottish Terrier separately. Then, I will add those weights together to get the total combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [24]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

"Thought: To find the combined weight of a Border Collie and a Scottish Terrier, I need to first find the average weight of each breed and then add them together. I'll start with the Border Collie.\nAction: average_dog_weight: Border Collie\nPAUSE"

In [28]:
next_prompt=f"Observation: {average_dog_weight('Border Collie')}"
next_prompt

'Observation: a Border Collies average weight is 37 lbs'

In [29]:
abot(next_prompt)

'Thought: Now that I know the average weight of a Border Collie is 37 lbs, I need to find the average weight of a Scottish Terrier to calculate the combined weight.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [30]:
next_prompt=f"Observation: {average_dog_weight('Scottish Terrier')}"
next_prompt

'Observation: Scottish Terriers average 20 lbs'

In [31]:
abot(next_prompt)

'Thought: With the average weight of a Border Collie being 37 lbs and a Scottish Terrier being 20 lbs, I can now calculate their combined weight.\nAction: calculate: 37 + 20\nPAUSE'

In [32]:
next_prompt = f"Observation: {calculate("37+20")}"
next_prompt

'Observation: 57'

In [33]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Automate the Whole Process

In [47]:
import re
# action_re = re.compile(r"Action: (.*)\nPAUSE")
action_re = re.compile(r"Action: (\w+): (.*)\nPAUSE")

In [55]:
def query(question, max_iterations=5):
    i=0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_iterations:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = action_re.findall(result)
        if actions:
            action, action_input = actions[0]
        else:
            return
        print(action,action_input)
        if action not in known_actions:
            raise Exception(f"Unknown action: {action}")
        print(f"---Executing action {action} with input {action_input}---")
        observation = known_actions[action](action_input)   
        print(f"Observation: {observation}")
        next_prompt = f"Observation: {observation}"
        
        
query("How much does a toy poodle weigh")


Thought: I should look up the dog's weight using average_dog_weight for a Toy Poodle.
Action: average_dog_weight: Toy Poodle
PAUSE
average_dog_weight Toy Poodle
---Executing action average_dog_weight with input Toy Poodle---
Observation: a toy poodles average weight is 7 lbs
Answer: A Toy Poodle weighs 7 lbs.


In [56]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: To find the combined weight of a Border Collie and a Scottish Terrier, I need to find the average weight of each breed and then add them together. I'll start by finding the average weight of a Border Collie.

Action: average_dog_weight: Border Collie
PAUSE
average_dog_weight Border Collie
---Executing action average_dog_weight with input Border Collie---
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I know the average weight of a Border Collie is 37 lbs, I need to find the average weight of a Scottish Terrier to calculate the combined weight.

Action: average_dog_weight: Scottish Terrier
PAUSE
average_dog_weight Scottish Terrier
---Executing action average_dog_weight with input Scottish Terrier---
Observation: Scottish Terriers average 20 lbs
Thought: With the average weight of a Border Collie being 37 lbs and a Scottish Terrier being 20 lbs, I can now calculate their combined weight.

Action: calculate: 37 + 20
PAUSE
calculate 37 + 20
---Executing a

In [59]:
question = """I have 3 dogs, a border collie, toy poodles and a scottish terrier. \
What is their combined weight and also average weight of the dogs upto 2 decimals places"""
query(question)

Thought: To find the combined and average weight of the three dogs, I need to first find the average weight of each breed: Border Collie, Toy Poodle, and Scottish Terrier. Then, I will add these weights together to get the combined weight. Finally, I will divide the combined weight by 3 to get the average weight of the dogs. I will start with the Border Collie.
Action: average_dog_weight: Border Collie
PAUSE
average_dog_weight Border Collie
---Executing action average_dog_weight with input Border Collie---
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I know the average weight of a Border Collie is 37 lbs, I need to find the average weight of a Toy Poodle next.
Action: average_dog_weight: Toy Poodle
PAUSE
average_dog_weight Toy Poodle
---Executing action average_dog_weight with input Toy Poodle---
Observation: a toy poodles average weight is 7 lbs
Thought: With the average weight of a Toy Poodle known to be 7 lbs, the next step is to find the average weight o